In [3]:
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
import yfinance as yf
from dataclasses import dataclass
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Validate environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")


@dataclass
class AnalysisConfig:
    """Configuration for analysis parameters"""

    period_years: int = 5
    interval: str = "1wk"  # 1d, 1wk, 1mo
    moving_averages: List[int] = None
    rsi_period: int = 14
    volume_ma_period: int = 20

    def __post_init__(self):
        if self.moving_averages is None:
            self.moving_averages = [50, 200]  # Default MAs


class FinancialMetrics:
    """Calculate financial metrics and ratios"""

    @staticmethod
    def calculate_ratios(financial_data: pd.DataFrame) -> Dict:
        try:
            current_ratio = (
                financial_data["Total Current Assets"]
                / financial_data["Total Current Liabilities"]
            )
            debt_to_equity = (
                financial_data["Total Liabilities"]
                / financial_data["Total Stockholder Equity"]
            )
            return {
                "current_ratio": current_ratio.iloc[-1],
                "debt_to_equity": debt_to_equity.iloc[-1],
                "quick_ratio": (
                    financial_data["Total Current Assets"] - financial_data["Inventory"]
                ).iloc[-1]
                / financial_data["Total Current Liabilities"].iloc[-1],
                "roe": (
                    financial_data["Net Income"]
                    / financial_data["Total Stockholder Equity"]
                ).iloc[-1],
            }
        except Exception as e:
            logger.error(f"Error calculating financial ratios: {e}")
            return {}


class TechnicalAnalysis:
    """Technical analysis calculations"""

    @staticmethod
    def calculate_rsi(data: pd.DataFrame, period: int = 14) -> pd.Series:
        delta = data["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def calculate_support_resistance(
        data: pd.DataFrame, window: int = 20
    ) -> Tuple[float, float]:
        rolling_min = data["Low"].rolling(window=window).min()
        rolling_max = data["High"].rolling(window=window).max()
        return rolling_min.iloc[-1], rolling_max.iloc[-1]


class ComprehensivePriceTool(BaseTool):
    name: str = "Comprehensive Price Analysis"
    description: str = "Get detailed historical price data with technical indicators"

    def __init__(self, config: AnalysisConfig):
        super().__init__()
        self._config = config  # Use _config instead of config

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(
                period=f"{self._config.period_years}y", interval=self._config.interval
            )

            if history.empty:
                return "No price data available for this ticker"

            # Calculate technical indicators
            for ma in self._config.moving_averages:
                history[f"MA_{ma}"] = history["Close"].rolling(window=ma).mean()

            history["RSI"] = TechnicalAnalysis.calculate_rsi(
                history, self._config.rsi_period
            )
            history["Volume_MA"] = (
                history["Volume"].rolling(window=self._config.volume_ma_period).mean()
            )

            support, resistance = TechnicalAnalysis.calculate_support_resistance(
                history
            )

            analysis_summary = {
                "price_data": history.to_csv(),
                "current_price": history["Close"].iloc[-1],
                "support_level": support,
                "resistance_level": resistance,
                "rsi": history["RSI"].iloc[-1],
                "volume_trend": (
                    "Up"
                    if history["Volume"].iloc[-1] > history["Volume_MA"].iloc[-1]
                    else "Down"
                ),
            }

            return str(analysis_summary)
        except Exception as e:
            logger.error(f"Error in price analysis: {e}")
            return f"Error analyzing price data: {str(e)}"


class FinancialMetricsTool(BaseTool):
    name: str = "Financial Metrics Analysis"
    description: str = "Analyze comprehensive financial metrics and growth trends"

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)

            # Get various financial statements
            balance_sheet = stock.balance_sheet
            income_stmt = stock.income_stmt
            cash_flow = stock.cashflow

            # Calculate key metrics and growth rates
            metrics = {
                "balance_sheet": balance_sheet.to_csv(),
                "income_statement": income_stmt.to_csv(),
                "cash_flow": cash_flow.to_csv(),
                "key_ratios": FinancialMetrics.calculate_ratios(balance_sheet),
                "revenue_growth": income_stmt.loc["Total Revenue"].pct_change().mean(),
                "profit_margin": (
                    income_stmt.loc["Net Income"] / income_stmt.loc["Total Revenue"]
                ).mean(),
            }

            return str(metrics)
        except Exception as e:
            logger.error(f"Error in financial analysis: {e}")
            return f"Error analyzing financial data: {str(e)}"


class AdvancedAgents:
    def __init__(self, config: AnalysisConfig):
        self.config = config

    def financial_analyst(self) -> Agent:
        return Agent(
            role="Senior Financial Analyst",
            goal="Provide comprehensive financial analysis focusing on long-term trends and company health",
            backstory="""A veteran financial analyst with 20 years of experience in equity research. 
                     Specialized in identifying long-term value opportunities through deep fundamental analysis.""",
            tools=[FinancialMetricsTool()],
            verbose=True,
        )

    def technical_analyst(self) -> Agent:
        return Agent(
            role="Technical Analysis Expert",
            goal="Analyze long-term price patterns and technical indicators to identify major trends",
            backstory="""A technical analysis expert with expertise in pattern recognition and trend analysis. 
                     Known for identifying major market turning points using multi-timeframe analysis.""",
            tools=[ComprehensivePriceTool(self.config)],
            verbose=True,
        )


def create_analysis_crew(ticker: str, config: AnalysisConfig = AnalysisConfig()) -> str:
    try:
        # Initialize agents
        agents = AdvancedAgents(config)

        # Create analysts
        financial_analyst = agents.financial_analyst()
        technical_analyst = agents.technical_analyst()

        # Define tasks with required expected_output field
        financial_analysis = Task(
            description=f"""Conduct a comprehensive {config.period_years}-year financial analysis of {ticker}.
                       Focus on: 
                       1. Long-term financial health trends
                       2. Growth metrics and sustainability
                       3. Risk factors and financial stability
                       4. Comparative industry analysis""",
            agent=financial_analyst,
            expected_output="""A detailed financial analysis report including:
                           - Key financial ratios and their trends
                           - Growth metrics and sustainability analysis
                           - Risk assessment
                           - Industry comparison metrics""",
        )

        technical_analysis = Task(
            description=f"""Analyze {config.period_years}-year price history for {ticker}.
                       Focus on:
                       1. Major trend identification
                       2. Support/resistance levels
                       3. Volume analysis
                       4. Technical indicator convergence/divergence""",
            agent=technical_analyst,
            expected_output="""A comprehensive technical analysis report including:
                           - Major trend analysis
                           - Support and resistance levels
                           - Volume analysis
                           - Technical indicator signals""",
        )

        # Create and configure crew
        crew = Crew(
            agents=[financial_analyst, technical_analyst],
            tasks=[financial_analysis, technical_analysis],
            verbose=True,
            process=Process.sequential,
            manager_llm=ChatOpenAI(
                model_name="gpt-4o-mini",
                temperature=0.5,
                api_key=OPENAI_API_KEY,
            ),
        )

        # Execute analysis
        result = crew.kickoff(inputs={"company": ticker})
        return result

    except Exception as e:
        logger.error(f"Error in analysis execution: {e}")
        raise


if __name__ == "__main__":
    # Example usage
    try:
        # Configure analysis parameters
        config = AnalysisConfig(
            period_years=5,
            interval="1wk",
            moving_averages=[50, 100, 200],
            rsi_period=14,
            volume_ma_period=20,
        )

        # Run analysis
        result = create_analysis_crew("066570.KS", config)
        print(result)

    except Exception as e:
        logger.error(f"Analysis failed: {e}")

LLM value is None
LLM value is None


16:40:12 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


# Agent: Senior Financial Analyst
## Task: Conduct a comprehensive 5-year financial analysis of 066570.KS.
                       Focus on: 
                       1. Long-term financial health trends
                       2. Growth metrics and sustainability
                       3. Risk factors and financial stability
                       4. Comparative industry analysis


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:40:14 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
ERROR:__main__:Error calculating financial ratios: 'Total Current Assets'
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_26500\2755295678.py:164: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  "revenue_growth": income_stmt.loc["Total Revenue"].pct_change().mean(),
16:40:15 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Senior Financial Analyst
## Thought: I should gather comprehensive financial metrics and growth trends for 066570.KS to analyze the company's long-term financial health and sustainability.
## Using tool: Financial Metrics Analysis
## Tool Input: 
"{\"ticker\": \"066570.KS\"}"
## Tool Output: 
{'balance_sheet': ',2023-12-31,2022-12-31,2021-12-31,2020-12-31\r\nTreasury Shares Number,763176.0,763176.0,763176.0,763176.0\r\nPreferred Shares Number,17181299.0,17181299.0,17181299.0,17181300.0\r\nOrdinary Shares Number,162884638.0,162884638.0,162884638.0,162884638.0\r\nShare Issued,163647814.0,163647814.0,163652503.0,163647814.0\r\nNet Debt,4833248000000.0,4722827000000.0,3880037000000.0,4023446000000.0\r\nTotal Debt,14513834000000.0,12141422000000.0,10886975000000.0,10781815000000.0\r\nTangible Book Value,16711137070000.0,16451348070000.0,14701464000000.0,12212490070000.0\r\nInvested Capital,32899635070000.0,29951365070000.0,27076266000000.0,25271325070000.0\r\nWorking Capital,6180

...



# Agent: Senior Financial Analyst
## Final Answer: 
### Comprehensive Financial Analysis of 066570.KS

#### 1. Long-term Financial Health Trends
Over the past five years, 066570.KS has exhibited a significant increase in total assets, growing from approximately 48.20 trillion KRW in 2020 to 60.24 trillion KRW in 2023, indicative of robust asset accumulation. However, the company's net income has seen fluctuations, reflecting a decrease in profitability as net income fell from 2.66 trillion KRW in 2020 to 0.71 trillion KRW in 2023. 

#### Key Financial Ratios Trend:
- **Debt-to-Equity Ratio**: Increased from approximately **0.70** to **0.73**, indicating a higher reliance on debt financing.
- **Current Ratio**: Improved from **1.15** in 2020 to approximately **1.25** in 2023. This suggests enhanced short-term liquidity.
- **Return on Equity (ROE)**: Decreased from about **16.5%** in 2020 to **3.5%** in 2023, indicating declining profitability relative to shareholder equity.

#### 2. G

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:40:29 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
16:40:29 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Technical Analysis Expert
## Using tool: Comprehensive Price Analysis
## Tool Input: 
"{\"ticker\": \"066570.KS\"}"
## Tool Output: 
{'price_data': 'Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA_50,MA_100,MA_200,RSI,Volume_MA\r\n2020-01-20 00:00:00+09:00,66544.953125,66544.953125,66544.953125,66544.953125,0,0.0,0.0,,,,,\r\n2020-01-27 00:00:00+09:00,66544.9547728566,66544.9547728566,63458.81194281107,63555.25390625,2562624,0.0,0.0,,,,,\r\n2020-02-03 00:00:00+09:00,63073.04789654532,65001.88728176078,61819.30229615528,64905.4453125,3325995,0.0,0.0,,,,,\r\n2020-02-10 00:00:00+09:00,64616.11328125,65966.30072294777,63748.13564015858,64616.11328125,2367365,0.0,0.0,,,,,\r\n2020-02-17 00:00:00+09:00,64519.672412774205,67509.373227118,63458.81083349092,63748.13671875,3744172,0.0,0.0,,,,,\r\n2020-02-24 00:00:00+09:00,62494.38746895695,62687.27138089818,57865.17358236755,58250.94140625,3895394,0.0,0.0,,,,,\r\n2020-03-02 00:00:00+09:00,58154.50390625,59697.57532001451,57479

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:40:37 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler




# Agent: Technical Analysis Expert
## Final Answer: 
### Comprehensive Technical Analysis Report for 066570.KS

#### 1. Major Trend Analysis
The 5-year price history of 066570.KS reveals significant fluctuations with an overall downtrend. Prices have fallen from a peak of approximately 143,636 KRW in early 2021 to the current price of about 85,100 KRW. The stock has experienced multiple price surges and corrections, reflecting volatility commonly found in tech stocks. 

#### 2. Support and Resistance Levels
- **Support Level:** 82,300 KRW
- **Resistance Level:** 113,700 KRW

These levels indicate key price points where the stock has historically shown reactions. The support level represents a potential buying opportunity, while the resistance level indicates where selling pressure may increase.

#### 3. Volume Analysis
Recent volume trends indicate an upward movement, which is a positive signal. Increasing volume during rising prices suggests that buying interest is building. In cont